In [1]:
import pandas as pd

import numpy as np

import xlsxwriter, os

import matplotlib.pyplot as plt

%matplotlib inline

from openpyxl import load_workbook

from sklearn import preprocessing

In [2]:
# save sales by industry into excel files

def create_excel_fnc(path, df, sheetname):
    
    writer = pd.ExcelWriter(path)
    
    df.to_excel(writer, sheetname)
    
    writer.save()
    
def add_sheet_fnc(path, df,sheetname):
    
    book = load_workbook(path)
    
    writer = pd.ExcelWriter(path, engine='openpyxl')
    
    writer.book = book
    
    df.to_excel(writer, sheet_name=sheetname)
    
    writer.save()
    
    writer.close()

# Process MN Industry 2015 Dataset

In [18]:
# load MN Industry Dataset

MN_df = pd.read_excel(os.getcwd()+r'\MN_Data\MN_Industry2015.xls')

In [19]:
# drop unexpected rows

MN_df = MN_df[MN_df.COUNTY != 'NON-MINNESOTA CO']

In [20]:
# extract all industry names

Industry_Names = MN_df.INDUSTRY.unique()

In [21]:
Industry_Names[33] = '999 UNDESIGNATED or SUPPRESSED'

In [22]:
MN_df.head()

,YEAR,COUNTY,INDUSTRY,GROSS SALES,TAXABLE SALES,SALES TAX,USE TAX,TOTAL TAX,NUMBER
0,2015,AITKIN,236 CONSTRUCT -BUILDINGS,1501102,172758,15100,1314,16414,4
1,2015,AITKIN,238 CONSTRUCT -SPECIAL TRADES,16217060,2286540,157199,22557,179756,31
2,2015,AITKIN,311 MFG -FOOD,3502899,54989,3781,1,3782,4
3,2015,AITKIN,314 MFG -TEXTILE PROD MILLS,207331,131150,9016,0,9016,5
4,2015,AITKIN,321 MFG -WOOD PRODUCT,28025899,203773,14009,1507,15516,6


In [ ]:
# MN_df = MN_df[['COUNTY', 'INDUSTRY','GROSS SALES']]

# MN_df = MN_df.set_index('COUNTY')

# MN_df.head()

## Save MN Sales data for each industry by county

In [9]:
County_Names = MN_df.COUNTY.unique().tolist()

In [11]:
for industry_name in Industry_Names:
    
    df = MN_df[MN_df.INDUSTRY == industry_name]
    
    Industry_County = df.COUNTY.tolist()      

    Industry_County_Sales = df[['COUNTY', 'GROSS SALES']]

    Industry_County_Sales.set_index('COUNTY', inplace=True)

    path1 = r'C:\Users\PL_Dell3668_One\Desktop\MN_Industry\Corr_Industry\Industry_Sales_Data\Industry_Sales.xlsx'
    
#     add_sheet_fnc(path1, Industry_County_Sales,industry_name)

# Load our raw data by County

In [8]:
# load our data

Raw_Data = pd.ExcelFile(r'C:\Users\PL_Dell3668_One\Desktop\MN_Industry\Corr_Industry\Raw_Data.xlsx')

Industry_Sales_Data = pd.ExcelFile(r'C:\Users\PL_Dell3668_One\Desktop\MN_Industry\Corr_Industry\Industry_Sales_Data\Industry_Sales.xlsx')

In [ ]:
Industry_Sales_Data

# Process Indices

In [10]:
sheetnames = Raw_Data.sheet_names

sheetnames.remove('Nat_Frn_Popn')

indices_names = sheetnames[0:8]

In [46]:
for industry_name in Industry_Names:
    
    writer = pd.ExcelWriter(industry_name+'.xlsx', engine='xlsxwriter')
    
    sales_df = MN_df[MN_df.INDUSTRY == industry_name]
    
    Industry_County = sales_df.COUNTY.tolist()
    
    path_raw = r'C:\Users\PL_Dell3668_One\Desktop\MN_Industry\Corr_Industry\Industry_Raw_Data\Indices_Industry'
        
    writer_raw = pd.ExcelWriter(path_raw+'\\'+industry_name+'.xlsx', engine='xlsxwriter')
    
    path_corr = r'C:\Users\PL_Dell3668_One\Desktop\MN_Industry\Corr_Industry\Industry_Correlation_Data\Indices_Industry'
        
    writer_corr = pd.ExcelWriter(path_corr+'\\'+industry_name+'.xlsx', engine='xlsxwriter')

    industry_sales_df = sales_df[['COUNTY', 'GROSS SALES']]
    
    industry_sales_df = industry_sales_df.set_index('COUNTY')
    
    industry_sales_df = industry_sales_df.astype('float64')
    
    for sheetname in indices_names:
        
        raw_df = Raw_Data.parse(sheetname)
        
        raw_df.set_index('County', inplace=True)
    
        raw_df = raw_df.loc[Industry_County]
        
        raw_df.to_excel(writer_raw, sheet_name=sheetname)

        corr_df = pd.DataFrame()       

        corr_df['Correlation Coefficient'] = raw_df[raw_df.columns[:]].apply(lambda x: x.corr(industry_sales_df['GROSS SALES']))
                
        corr_df.to_excel(writer_corr, sheet_name=sheetname)       

C:\Anaconda\lib\site-packages\numpy\lib\function_base.py:3159: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Anaconda\lib\site-packages\numpy\lib\function_base.py:3093: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)


# Process Categories

In [38]:
Category_names = sheetnames[9:]

In [37]:
for industry_name in Industry_Names:
    
#     writer = pd.ExcelWriter(industry_name+'.xlsx', engine='xlsxwriter')
    
    sales_df = MN_df[MN_df.INDUSTRY == industry_name]
    
    industry_sales_df = sales_df[['COUNTY', 'GROSS SALES']]
    
    industry_sales_df = industry_sales_df.set_index('COUNTY')
    
    industry_sales_df = industry_sales_df.astype('float64')
    
    Industry_County = sales_df.COUNTY.tolist()
    
    path_raw = r'C:\Users\PL_Dell3668_One\Desktop\MN_Industry\Corr_Industry\Industry_Raw_Data\Categories_Industry'
        
    writer_raw = pd.ExcelWriter(path_raw+'\\'+industry_name+'.xlsx', engine='xlsxwriter')
    
    path_corr = r'C:\Users\PL_Dell3668_One\Desktop\MN_Industry\Corr_Industry\Industry_Correlation_Data\Categories_Industry'
        
    writer_corr = pd.ExcelWriter(path_corr+'\\'+industry_name+'.xlsx', engine='xlsxwriter')
    
    for sheetname in Category_names:
        
        df = Raw_Data.parse(sheetname)
        
        variable_names = df.loc[0][1:]
        
        df.set_index('County', inplace=True)
    
        df = df.loc[Industry_County]
        
        df = df.astype('float64')
        
        raw_df = df.reset_index()

        raw_df.loc[-1] = variable_names

        raw_df.index = raw_df.index+1

        raw_df = raw_df.sort_index()
        
#         raw_df = raw_df.astype('float')
        
#         raw_df.to_excel(writer_raw, sheet_name=sheetname)

        corr_df = pd.DataFrame() 
        
        corr_df['Variable Name'] = variable_names

        corr_df['Correlation Coefficient'] = df[df.columns[:]].apply(lambda x: x.corr(industry_sales_df['GROSS SALES']))
                
        corr_df.to_excel(writer_corr, sheet_name=sheetname)       

C:\Anaconda\lib\site-packages\numpy\lib\function_base.py:3159: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Anaconda\lib\site-packages\numpy\lib\function_base.py:3093: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)
